In [21]:
import nltk
from nltk.corpus import stopwords
import string
from string import punctuation
import os
from os import listdir
from collections import Counter

In [1]:
def text_to_tokens(filename):
 '''Opens the input text file and
 returns a list of all of its words.'''
 file = open(filename, 'r')
 text = file.read()
 file.close()
 tokens = text.split()
 return tokens

In [16]:
# go up one level of the directory to be able to get into the data folder
cd ..

/Users/kamilapalys/Desktop/school/data450/capstone


In [17]:
# Ensure we're in the correct directory to be able to navigate to the data folder
pwd

'/Users/kamilapalys/Desktop/school/data450/capstone'

In [24]:
# example of using the function

filepath = 'data/text/cnn_trump.txt'
text = text_to_tokens(filepath)
len(text)
text[:15]

['Donald',
 'Trump',
 'faces',
 'more',
 'than',
 '30',
 'counts',
 'related',
 'to',
 'business',
 'fraud',
 'in',
 'an',
 'indictment',
 'from']

In [32]:
# playing around with stopwords, going through example of removing them

stopwords_list = stopwords.words('english')
print(stopwords_list)

token_ex = ['was', 'she', 'ever', 'going',
             'to', 'realize', 'they', 'were', 'laughing', 'at', 'her']

new = []

for token in token_ex:
    if token not in stopwords_list:
        new.append(token)

#print(new)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
def clean_tokens(tokens):
    '''Takes in a list of tokens and cleans them by converting
    to lowercase, removing punctuation, removing stopwords, 
    and stemming. Returns the new list of tokens.'''
    new_tokens = []
    for token in tokens:
        token.lower()
        if token # is a stopword: pass, else: if not in new token list: append


In [23]:
# how all the text files will be looped through

directory = 'data/text/'
for filename in os.listdir(directory):
    filepath = str(os.path.join(directory, filename))
    # use preprocessing functions
    # somehow append each list of tokens (each article) to a dataframe
    

data/text/nyp_affirmative.txt
data/text/bbc_tanks.txt
data/text/cnn_hamas.txt
data/text/wsj_trump.txt
data/text/wp_affirmative.txt
data/text/bbc_balloon.txt
data/text/wsj_affirmative.txt
data/text/nyt_trump.txt
data/text/wp_balloon.txt
data/text/cnn_pentagon.txt
data/text/cnn_tanks.txt
data/text/bbc_hamas.txt
data/text/nbc_tanks.txt
data/text/nyp_hamas.txt
data/text/.DS_Store
data/text/cnn_balloon.txt
data/text/fox_pentagon.txt
data/text/wp_biden.txt
data/text/nyp_tanks.txt
data/text/nbc_hamas.txt
data/text/abc_balloon.txt
data/text/nyt_pentagon.txt
data/text/fox_tanks.txt
data/text/wp_trump.txt
data/text/fox_hamas.txt
data/text/nyp_balloon.txt
data/text/abc_hamas.txt
data/text/abc_santos.txt
data/text/nyp_santos.txt
data/text/fox_affirmative.txt
data/text/wsj_biden.txt
data/text/nyt_biden.txt
data/text/abc_tanks.txt
data/text/nbc_affirmative.txt
data/text/nyt_hamas.txt
data/text/wsj_tanks.txt
data/text/wp_pentagon.txt
data/text/nyp_pentagon.txt
data/text/bbc_trump.txt
data/text/nyt_af

In [ ]:
# takes all docs and makes them into a dataframe
# make sure that docs is just a list of all the tokens from all articles
# first make one list of tokens for each article and then add all those lists up
# mode is one of "binary", "count", "tfidf", "freq"
tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    analyzer=None,
    **kwargs
)

Xtrain = tokenizer.texts_to_matrix(docs, mode='binary')
print(Xtrain.shape)